In [1]:
# %%cmd
# pip install nltk
# pip install datasets
# pip install tokenizers
# pip install evaluate
# pip install rouge_score
# pip install sentencepiece
# pip install huggingface_hub

Microsoft Windows [Version 10.0.22631.3527]
(c) Microsoft Corporation. All rights reserved.

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install nltk

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install datasets

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install tokenizers

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install evaluate

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install rouge_score

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install sentencepiece

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>pip install huggingface_hub

(llama3-test) C:\Users\ryanw\PycharmProjects\llama3-test>

In [2]:
import nltk
import evaluate
import numpy as np
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

In [3]:
# Load the tokenizer, model, and data collator
MODEL_NAME = "google/flan-t5-small"

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
# Acquire the training data from Hugging Face
DATA_NAME = "yahoo_answers_qa"
yahoo_answers_qa = load_dataset(DATA_NAME)

In [5]:
yahoo_answers_qa = yahoo_answers_qa["train"].train_test_split(test_size=0.3)
# yahoo_answers_qa["train"] = yahoo_answers_qa["train"].select(range(100))
# yahoo_answers_qa["test"] = yahoo_answers_qa["test"].select(range(10))


In [6]:
# Check the length of the data and its structure
yahoo_answers_qa
# 61,153 examples trained in 2 hours over 3 epochs

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 61153
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'nbestanswers', 'main_category'],
        num_rows: 26209
    })
})

In [7]:
from datasets import load_dataset
yahoo_answers_qa = load_dataset("csv", data_files="dataset.csv")
yahoo_answers_qa

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 2
    })
})

In [8]:
# We prefix our tasks with "answer the question"
prefix = "Please answer this question: "

# Define the preprocessing function

def preprocess_function(examples):
   """Add prefix to the sentences, tokenize the text, and set the labels"""
   # The "inputs" are the tokenized answer:
   inputs = [prefix + doc for doc in examples["question"]]
   model_inputs = tokenizer(inputs, max_length=128, truncation=True)
  
   # The "labels" are the tokenized outputs:
   labels = tokenizer(text_target=examples["answer"], 
                      max_length=512,         
                      truncation=True)

   model_inputs["labels"] = labels["input_ids"]
   return model_inputs

In [9]:
# Map the preprocessing function across our dataset
tokenized_dataset = yahoo_answers_qa.map(preprocess_function, batched=True)

In [10]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

In [11]:
tokenized_dataset["train"]['labels'][0]

[1919, 1]

In [6]:
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

In [7]:
def compute_metrics(eval_preds):
   preds, labels = eval_preds

   # decode preds and labels
   labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
   decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
   decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

   # rougeLSum expects newline after each sentence
   decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
   decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

   result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  
   return result

In [15]:
# Global Parameters
L_RATE = 3e-4
BATCH_SIZE = 8
PER_DEVICE_EVAL_BATCH = 4
WEIGHT_DECAY = 0.01
SAVE_TOTAL_LIM = 3
NUM_EPOCHS = 3

# Set up training arguments
training_args = Seq2SeqTrainingArguments(
   output_dir="./results",
   # evaluation_strategy="epoch",
   learning_rate=L_RATE,
   per_device_train_batch_size=BATCH_SIZE,
   # per_device_eval_batch_size=PER_DEVICE_EVAL_BATCH,
   weight_decay=WEIGHT_DECAY,
   save_total_limit=SAVE_TOTAL_LIM,
   num_train_epochs=NUM_EPOCHS,
   predict_with_generate=True,
   push_to_hub=False
)

In [16]:
# trainer = Seq2SeqTrainer(
#    model=model,
#    args=training_args,
#    train_dataset=tokenized_dataset["train"],
#    eval_dataset=tokenized_dataset["test"],
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics
# )

# trainer = Seq2SeqTrainer(
#    model=model,
#    args=training_args,
#    train_dataset=tokenized_dataset["train"],
#    eval_dataset=tokenized_dataset["test"],
#    tokenizer=tokenizer,
#    data_collator=data_collator,
#    compute_metrics=compute_metrics
# )

trainer = Seq2SeqTrainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_dataset["train"],
   # eval_dataset=tokenized_dataset["test"],
   tokenizer=tokenizer,
   data_collator=data_collator,
   # compute_metrics=compute_metrics
)

In [17]:
trainer.train()

Step,Training Loss


TrainOutput(global_step=3, training_loss=0.4493695894877116, metrics={'train_runtime': 0.4602, 'train_samples_per_second': 13.037, 'train_steps_per_second': 6.519, 'total_flos': 37032873984.0, 'train_loss': 0.4493695894877116, 'epoch': 3.0})